In [1]:
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data
import os

def load_and_process_data(prefix, folder):
    node_file = f'{folder}/{prefix}_nodes.csv'
    link_file = f'{folder}/{prefix}_links.csv'

    node_data = pd.read_csv(node_file)
    link_data = pd.read_csv(link_file)

    # Assuming the ground truth labels are in the same format
    labels = node_data['ground_truth'].values
    features = node_data.drop(columns=['ground_truth'])

    node_features = features[['atom_type', 'residue_type', 'radius', 'voromqa_sas_potential', 'residue_mean_sas_potential', 'residue_sum_sas_potential', 'residue_size', 'sas_area', 'voromqa_sas_energy', 'voromqa_depth', 'voromqa_score_a', 'voromqa_score_r', 'volume', 'volume_vdw', 'ufsr_a1', 'ufsr_a2', 'ufsr_c2', 'ufsr_c3', 'ev28', 'ev56']]
    link_features = link_data[['atom_index1', 'atom_index2','area', 'boundary', 'distance', 'voromqa_energy', 'seq_sep_class', 'covalent_bond', 'hbond']]

    edge_index = torch.tensor(np.array([link_features['atom_index1'].values, link_features['atom_index2'].values]), dtype=torch.long)

    self_links = torch.arange(0, len(node_features))
    edge_index = torch.cat([edge_index, torch.stack([self_links, self_links])], dim=1)
    edge_index = torch.cat([edge_index, edge_index[[1, 0], :]], dim=1)  # Add reverse direction

    node_features_tensor = torch.tensor(node_features.values, dtype=torch.float)
    labels_tensor = torch.tensor(labels, dtype=torch.float)

    data = Data(x=node_features_tensor, edge_index=edge_index, y=labels_tensor)

    return data

candidate_pairs_file = 'holo/candidate_pairs.txt'
candidate_pairs = pd.read_csv(candidate_pairs_file, delim_whitespace=True)

graphs = {}
for index, row in candidate_pairs.iterrows():
    holo_prefix = f"{row['holo_pdb_id']}_{row['holo_chain_id']}"
    graphs[holo_prefix] = load_and_process_data(holo_prefix, 'holo')

save_dir = 'sh'
os.makedirs(save_dir, exist_ok=True)

for prefix, graph in graphs.items():
    save_path = os.path.join(save_dir, f'{prefix}_graph.pt')
    torch.save(graph, save_path)

print(f"All HOLO graphs have been saved in the directory: {save_dir}")

All HOLO graphs have been saved in the directory: sh


In [11]:
import os
import pandas as pd
import torch
from torch_geometric.data import DataLoader, Batch
import matplotlib.pyplot as plt

# Function to load data pairs
def load_data_pairs(apo_folder='sg', holo_folder='sh', pairs_file='apo/candidate_pairs.txt'):
    pairs = pd.read_csv(pairs_file, delim_whitespace=True)
    data_pairs = []
    for index, row in pairs.iterrows():
        apo_prefix = f"{row['apo_pdb_id']}_{row['apo_chain_id']}"
        holo_prefix = f"{row['holo_pdb_id']}_{row['holo_chain_id']}"
        apo_graph_path = os.path.join(apo_folder, f'{apo_prefix}_graph.pt')
        holo_graph_path = os.path.join(holo_folder, f'{holo_prefix}_graph.pt')
        if os.path.exists(apo_graph_path) and os.path.exists(holo_graph_path):
            apo_graph = torch.load(apo_graph_path)
            holo_graph = torch.load(holo_graph_path)
            data_pairs.append((apo_graph, holo_graph))
    return data_pairs

# Function to collate graphs for DataLoader
def collate_graphs(data_list):
    apo_data_list, holo_data_list = zip(*data_list)
    apo_batch = Batch.from_data_list(apo_data_list)
    holo_batch = Batch.from_data_list(holo_data_list)
    return apo_batch, holo_batch

# Load data pairs and set up DataLoader
data_pairs = load_data_pairs()
train_loader = DataLoader(data_pairs, batch_size=32, shuffle=True, collate_fn=collate_graphs)

# Inspect the first batch from DataLoader
first_batch = next(iter(train_loader))
apo_batch, holo_batch = first_batch

# Print the shapes of the tensors in the first batch
print("APO Batch:")
print("x shape:", apo_batch.x.shape)
print("edge_index shape:", apo_batch.edge_index.shape)
print("batch shape:", apo_batch.batch.shape)
print("y shape:", apo_batch.y.shape)

print("\nHOLO Batch:")
print("x shape:", holo_batch.x.shape)
print("edge_index shape:", holo_batch.edge_index.shape)
print("batch shape:", holo_batch.batch.shape)
print("y shape:", holo_batch.y.shape)

/home/flahaari/.local/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


APO Batch:
x shape: torch.Size([75498, 20])
edge_index shape: torch.Size([2, 1227630])
batch shape: torch.Size([75498])
y shape: torch.Size([75498])

HOLO Batch:
x shape: torch.Size([75486, 20])
edge_index shape: torch.Size([2, 1230080])
batch shape: torch.Size([75486])
y shape: torch.Size([75486])


In [10]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data, DataLoader, Batch
import torch_geometric.nn as geom_nn
import gvp
import matplotlib.pyplot as plt
from tqdm import tqdm

# Define the Siamese GVP Model
class SiameseGVPModel(nn.Module):
    def __init__(self, node_in_dims):
        super(SiameseGVPModel, self).__init__()
        self.gvp1 = gvp.GVP(node_in_dims, (64, 0), vector_gate=True, activations=(F.relu, None))
        self.gvp2 = gvp.GVP((64, 0), (1, 0))

    def forward(self, apo_data, holo_data):
        apo_x = self.process_structure(apo_data)
        holo_x = self.process_structure(holo_data)
        combined = torch.cat([apo_x, holo_x], dim=1)
        combined = F.relu(self.fc1(combined))
        combined = self.fc2(combined)
        return combined

    def process_structure(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gvp1(x)
        x = self.gvp2(x)
        if hasattr(data, 'batch') and data.batch is not None:
            x = geom_nn.global_mean_pool(x[0], data.batch)  # Apply pooling only if batch tensor is present
        else:
            x = x.mean(dim=0, keepdim=True)  # Default to mean pooling if batch tensor is not present
        return x

def collate_graphs(data_list):
    apo_data_list, holo_data_list = zip(*data_list)
    apo_batch = Batch.from_data_list(apo_data_list)
    holo_batch = Batch.from_data_list(holo_data_list)
    return apo_batch, holo_batch

# Load paired data
def load_data_pairs(apo_folder='sg', holo_folder='sh', pairs_file='apo/candidate_pairs.txt'):
    pairs = pd.read_csv(pairs_file, delim_whitespace=True)
    data_pairs = []

    for index, row in pairs.iterrows():
        apo_prefix = f"{row['apo_pdb_id']}_{row['apo_chain_id']}"
        holo_prefix = f"{row['holo_pdb_id']}_{row['holo_chain_id']}"

        apo_graph_path = os.path.join(apo_folder, f'{apo_prefix}_graph.pt')
        holo_graph_path = os.path.join(holo_folder, f'{holo_prefix}_graph.pt')

        if os.path.exists(apo_graph_path) and os.path.exists(holo_graph_path):
            apo_graph = torch.load(apo_graph_path)
            holo_graph = torch.load(holo_graph_path)
            data_pairs.append((apo_graph, holo_graph))

    return data_pairs

def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch_data in loader:
        # Unpack the batch data
        apo_batch, holo_batch = batch_data
        apo_batch = apo_batch.to(device)
        holo_batch = holo_batch.to(device)

        optimizer.zero_grad()

        # Pass both apo and holo data to the model
        output = model(apo_batch, holo_batch)

        # Assuming the target labels are in apo_batch.y
        loss = criterion(output.squeeze(), apo_batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

# Load data pairs
data_pairs = load_data_pairs()

# Set up DataLoader
train_loader = DataLoader(data_pairs, batch_size=32, shuffle=True, collate_def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for apo_batch, holo_batch in loader:
        apo_batch, holo_batch = apo_batch.to(device), holo_batch.to(device)
        optimizer.zero_grad()
        apo_output = model(apo_batch)
        holo_output = model(holo_batch)
        output = torch.cat([apo_output, holo_output], dim=1)  # Concatenate the outputs
        # Assume that the ground truth labels are the same for both apo and holo data
        loss = criterion(output, apo_batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)fn=collate_graphs)

# Model setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseGVPModel(node_in_dims=(20, 0)).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
num_epochs = 10
loss_history = []

for epoch in tqdm(range(num_epochs), desc="Epochs"):
    loss = train(model, train_loader, optimizer, criterion, device)
    loss_history.append(loss)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}')

# Plotting the training loss
plt.figure(figsize=(10, 6))
plt.plot(loss_history, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

/home/flahaari/.local/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
Epochs:   0%|                                            | 0/10 [00:03<?, ?it/s]


RuntimeError: Expected index [70727] to be smaller than self [32] apart from dimension 0 and to be smaller size than src [1]

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import DataLoader, Batch
from tqdm import tqdm
import matplotlib.pyplot as plt

class SiameseGCNModel(nn.Module):
    def __init__(self, node_in_dims, node_out_dims=64):
        super(SiameseGCNModel, self).__init__()
        self.gcn1 = GCNConv(node_in_dims, node_out_dims)
        self.gcn2 = GCNConv(node_out_dims, node_out_dims)

    def forward(self, apo_data, holo_data):
        apo_x = self.process_structure(apo_data)
        holo_x = self.process_structure(holo_data)
        combined = torch.cat([apo_x, holo_x], dim=1)
        return combined

    def process_structure(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.gcn1(x, edge_index))
        x = self.gcn2(x, edge_index)
        x = global_mean_pool(x, data.batch)  # Apply global mean pooling
        return x

def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for apo_batch, holo_batch in loader:
        apo_batch = apo_batch.to(device)
        holo_batch = holo_batch.to(device)
        optimizer.zero_grad()
        output = model(apo_batch, holo_batch)
        loss = criterion(output, apo_batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Load data pairs and setup DataLoader
data_pairs = load_data_pairs()
train_loader = DataLoader(data_pairs, batch_size=32, shuffle=True, collate_fn=collate_graphs)

# Model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseGCNModel(node_in_dims=20).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
num_epochs = 10
loss_history = []

for epoch in tqdm(range(num_epochs), desc="Epochs"):
    loss = train(model, train_loader, optimizer, criterion, device)
    loss_history.append(loss)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss:.4f}')

# Plotting the training loss
plt.figure(figsize=(10, 6))
plt.plot(loss_history, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Epochs')
plt.legend()
plt.show()

NameError: name 'load_data_pairs' is not defined